In [47]:
import pandas as pd
rr = pd.read_parquet('../data/compound_results.parquet')
# aa = pd.read_csv('../data/compound/LARC_rzsm_historical_daily.csv')
# bb = pd.read_csv('../data/compound/LARC_rzsm_ssp245_daily.csv')
# aa = pd.concat([aa,bb])
# aa = aa.rename(columns={'Unnamed: 0':'date'})
# aa.columns = aa.columns.str.lower()
# aa.date = pd.to_datetime(aa.date)
# aa[aa.date.dt.year.between(*(1981, 2020))].drop(columns='date').quantile([0.02, 0.05, 0.1, 0.2])#.info()
rr

,event,threshold,center,ssp,date,access-cm2_day_total,access-cm2_duration_max,access-cm2_duration_mean,access-cm2_event_total,access-cm2_sequence_total,...,noresm2-mm_day_total,noresm2-mm_duration_max,noresm2-mm_duration_mean,noresm2-mm_event_total,noresm2-mm_sequence_total,taiesm1_day_total,taiesm1_duration_max,taiesm1_duration_mean,taiesm1_event_total,taiesm1_sequence_total
0,CDHE,spi<-1_tasmax>32.22,AFRC,ssp126,2015,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,CDHE,spi<-1_tasmax>32.22,AFRC,ssp126,2016,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,CDHE,spi<-1_tasmax>32.22,AFRC,ssp126,2017,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,CDHE,spi<-1_tasmax>32.22,AFRC,ssp126,2018,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,CDHE,spi<-1_tasmax>32.22,AFRC,ssp126,2019,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,CWHE2,rzsm>0.9_tasmax>32.22,WSTF,ssp245,2095,19.0,8.0,6.33,19.0,3.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4496,CWHE2,rzsm>0.9_tasmax>32.22,WSTF,ssp245,2096,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4497,CWHE2,rzsm>0.9_tasmax>32.22,WSTF,ssp245,2097,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4498,CWHE2,rzsm>0.9_tasmax>32.22,WSTF,ssp245,2098,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [39]:
import pandas as pd

pp = pd.read_parquet('../data/compound_raw.parquet')
sm = pp[(pp.center=='LARC') & (pp.variable=='rzsm') & (pp.ssp.isin(['ssp245','historical']))]
tm = pp[(pp.center=='LARC') & (pp.variable=='tasmax') & (pp.ssp.isin(['ssp245','historical']))]
ppp = sm[sm.date.dt.year.between(*(1981, 2020))].drop(columns=['center', 'variable', 'ssp', 'date']).quantile(0.3)#.info()
# YEAR = 2067
# sm = sm[sm.date.dt.year == YEAR].set_index('date').drop(columns=['center', 'variable', 'ssp'])
# tm = tm[tm.date.dt.year == YEAR].set_index('date').drop(columns=['center', 'variable', 'ssp'])

# rr = pd.read_parquet('../data/compound_results.parquet')
# rr = rr[(rr.center=='LARC') & (rr.event=='CDHE2') & (rr.threshold == 'rzsm<0.02_tasmax>32.22') & (rr.date ==YEAR)]
# rr.filter(regex='_day_total')

sm = sm.set_index('date').drop(columns=['center', 'variable', 'ssp'])
tm = tm.set_index('date').drop(columns=['center', 'variable', 'ssp'])

rr = pd.read_parquet('../data/compound_results.parquet')
rr = rr[(rr.center=='LARC') & (rr.event=='CDHE2') & (rr.threshold == 'rzsm<0.3_tasmax>32.22')]
# rr.filter(regex='_day_total')

from functools import reduce
dfs = [sm.dropna(axis=1, how='all').lt(ppp), tm.dropna(axis=1, how='all').gt(32.2222222)]
common_idx = reduce(lambda x, y: x.intersection(y), [df.index for df in dfs])
common_cols = set.intersection(*[set(df.columns) for df in dfs])
dfs_ = [df.loc[common_idx, sorted(common_cols)] for df in dfs]
import numpy as np
arr = np.stack([df.values for df in dfs_])
valid_count = np.sum(~np.isnan(arr), axis=0)
ones_count = np.sum(arr == 1, axis=0)
compound_vals = np.where(valid_count < 2, np.nan, (ones_count >= 2).astype(float))
df_out = pd.DataFrame(compound_vals, index=dfs_[0].index, columns=dfs_[0].columns)
df_out.sum()

access-cm2       4670.0
access-esm1-5    5371.0
bcc-csm2-mr         0.0
cmcc-esm2        3271.0
cnrm-cm6-1       5566.0
cnrm-esm2-1      5545.0
ec-earth3        5031.0
fgoals-g3        3615.0
gfdl-esm4        3595.0
giss-e2-1-g      4207.0
inm-cm4-8        3261.0
inm-cm5-0        3485.0
ipsl-cm6a-lr        0.0
kace-1-0-g       3605.0
miroc-es2l       3630.0
miroc6              0.0
mpi-esm1-2-hr    4435.0
mpi-esm1-2-lr    3659.0
mri-esm2-0       3654.0
noresm2-lm       4025.0
noresm2-mm       4009.0
dtype: float64

In [38]:
#0.2
yy = [2050, 2051]
aa = df_out[df_out.index.year.isin(yy)]
aa[aa.eq(1).any(axis=1)]#.sum().mean()
mm = 'gfdl-esm4'
print(rr[rr.date.isin(yy)].filter(regex='duration_mean').mean(axis=1))
display(rr[rr.date.isin(yy)].filter(regex=mm))
display(rr[rr.date.isin(yy)].filter(regex='duration_mean').T)
# display(aa[aa[mm]==1][mm])

2350    5.643333
2351    5.379444
dtype: float64


,gfdl-esm4_day_total,gfdl-esm4_duration_max,gfdl-esm4_duration_mean,gfdl-esm4_event_total,gfdl-esm4_sequence_total
2350,4.0,3.0,3.00,3.0,1.0
2351,43.0,16.0,5.25,42.0,8.0


,2350,2351
access-cm2_duration_mean,5.20,6.71
access-esm1-5_duration_mean,8.00,6.67
cmcc-cm2-sr5_duration_mean,NaN,NaN
cmcc-esm2_duration_mean,3.00,8.00
cnrm-cm6-1_duration_mean,6.00,3.40
cnrm-esm2-1_duration_mean,19.75,5.33
ec-earth3_duration_mean,8.00,7.62
fgoals-g3_duration_mean,2.33,4.00
gfdl-cm4_duration_mean,NaN,NaN
gfdl-esm4_duration_mean,3.00,5.25


In [34]:
#0.1
yy = [2050, 2051]
aa = df_out[df_out.index.year.isin(yy)]
aa[aa.eq(1).any(axis=1)]#.sum().mean()
mm = 'gfdl-esm4'
print(rr[rr.date.isin(yy)].filter(regex='duration_mean').mean(axis=1))
display(rr[rr.date.isin(yy)].filter(regex=mm))
display(rr[rr.date.isin(yy)].filter(regex='duration_mean').T)
# display(aa[aa[mm]==1][mm])

3850    4.257778
3851    5.011111
dtype: float64


,gfdl-esm4_day_total,gfdl-esm4_duration_max,gfdl-esm4_duration_mean,gfdl-esm4_event_total,gfdl-esm4_sequence_total
3850,0.0,0.0,0.0,0.0,0.0
3851,15.0,8.0,6.5,13.0,2.0


,3850,3851
access-cm2_duration_mean,5.75,6.00
access-esm1-5_duration_mean,6.82,6.11
cmcc-cm2-sr5_duration_mean,NaN,NaN
cmcc-esm2_duration_mean,0.00,7.17
cnrm-cm6-1_duration_mean,4.00,2.67
cnrm-esm2-1_duration_mean,18.00,7.67
ec-earth3_duration_mean,6.00,7.60
fgoals-g3_duration_mean,2.40,3.17
gfdl-cm4_duration_mean,NaN,NaN
gfdl-esm4_duration_mean,0.00,6.50


In [40]:
#0.3
yy = [2050, 2051]
aa = df_out[df_out.index.year.isin(yy)]
aa[aa.eq(1).any(axis=1)]#.sum().mean()
mm = 'gfdl-esm4'
print(rr[rr.date.isin(yy)].filter(regex='duration_mean').mean(axis=1))
display(rr[rr.date.isin(yy)].filter(regex=mm))
display(rr[rr.date.isin(yy)].filter(regex='duration_mean').T)
# display(aa[aa[mm]==1][mm])

850    5.368333
851    5.515000
dtype: float64


,gfdl-esm4_day_total,gfdl-esm4_duration_max,gfdl-esm4_duration_mean,gfdl-esm4_event_total,gfdl-esm4_sequence_total
850,10.0,4.0,3.5,7.0,2.0
851,55.0,16.0,5.0,50.0,10.0


,850,851
access-cm2_duration_mean,4.44,6.67
access-esm1-5_duration_mean,7.45,6.67
cmcc-cm2-sr5_duration_mean,NaN,NaN
cmcc-esm2_duration_mean,2.67,7.29
cnrm-cm6-1_duration_mean,4.17,4.00
cnrm-esm2-1_duration_mean,20.50,5.36
ec-earth3_duration_mean,6.50,7.00
fgoals-g3_duration_mean,2.33,4.00
gfdl-cm4_duration_mean,NaN,NaN
gfdl-esm4_duration_mean,3.50,5.00


In [44]:
rr[rr.date.isin(yy)].filter(regex='duration_mean').loc[850].values

array([ 4.44,  7.45,   nan,  2.67,  4.17, 20.5 ,  6.5 ,  2.33,   nan,
        3.5 ,  5.67,  0.  ,  6.57,  5.25,  5.  ,  3.  ,  4.8 ,  5.78,
        5.  ,  4.  ,   nan])

In [45]:
!python get_compound.py -i '../data/' -o '../data/compound_results.parquet' -e {' '.join(['CDHE2'])}

Saved to ../data/compound_results.parquet
